In [2]:
!pip install opendatasets --upgrade

In [3]:
import os
os.getcwd()

'/content'

In [4]:
import torch
import pandas as pd
import numpy as np
import imageio as iio
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from dataFunctions import *
from torch.utils.data import DataLoader
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import *
from tqdm import tqdm
import os
import torchvision
%load_ext autoreload

%autoreload 2

ModuleNotFoundError: No module named 'dataFunctions'

# Load in Kaggle Bird Dataset

In [1]:
import opendatasets as od
dataset_url = 'https://www.kaggle.com/datasets/gpiosenka/100-bird-species'
od.download(dataset_url)

ModuleNotFoundError: No module named 'opendatasets'

In [ ]:
df = pd.read_csv('train_info.csv')
df = pd.DataFrame(df.values[::10], columns = df.columns)
df.to_csv('training.csv')

In [ ]:
train_images = BirdImageDataset('training.csv')
test_image = BirdImageDataset('test_info.csv')

In [ ]:
train_loader = DataLoader(train_images, batch_size= 32, shuffle= True)
test_loader = DataLoader(test_image, batch_size= 32, shuffle= True)

In [ ]:
next(iter(train_loader))[1].shape

torch.Size([32, 525])

In [ ]:
birds = pd.read_csv(r'birds.csv', sep = ',')
birds[birds['data set'] == 'valid'].to_csv('valid_info.csv')

In [ ]:
label_map = sorted(dict(zip(birds['class id'].unique(), birds['labels'].unique())).items())
label_map

In [ ]:
label = torch.zeros(525)
label[int(birds['class id'].iloc[8000])] = 1

In [ ]:
img_labels = pd.read_csv('train_info.csv')
def change_path(filepath):
        els = filepath.split('/')
        els[1] = 'PARAKETT  AUKLET'
        return os.path.join(*els)

img_labels.loc[img_labels['labels'] == 'PARAKETT  AKULET', 'filepaths'] =  img_labels.loc[img_labels['labels'] == 'PARAKETT  AKULET', 'filepaths'].map(change_path)


In [ ]:
#Definition of hyperparameters
n_iters = 4500
num_epochs = 100

# Option 1: passing weights param as string
model = torchvision.models.resnet18(weights = 'IMAGENET1K_V1')
model_requires_grad_params = []
for param in model.parameters():
    param.requires_grad = False

class FFClassifier(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.fc1 = nn.Linear(in_features, out_features)
        self.drop = nn.Dropout(0.2)

    def forward(self, x):
        x = self.drop(x)
        x = self.fc1(x)

        x = F.log_softmax(x, dim=1)
        return x

model.fc = FFClassifier(in_features=512, out_features=525)


for param in model.parameters():

    if param.requires_grad == True:
        model_requires_grad_params.append(param)

model.cuda()
print(model)

# Cross Entropy Loss
l = nn.CrossEntropyLoss()

# Optimizer
learning_rate = 0.001
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.1)

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
model(next(iter(train_loader))[0].to(device)).to(device).shape

torch.Size([32, 525])

In [ ]:
tot_epoch_loss = []
model.to(device)
for epoch in tqdm(range(num_epochs)):
    print(f'EPOCH: {epoch}/{num_epochs}')
    model.train()
    batch_loss = []
    for idx, batch in enumerate(train_loader):


        train_x, train_y = batch
        #print(train_y.shape)
        optimizer.zero_grad()

        output = model(train_x.to(device))#.unsqueeze(1)
        #print(output.shape)
        loss = l(output, train_y.to(device))

        loss.backward()
        batch_loss.append(loss.item())

        if idx%100 == 0:
            print(f'BATCH {idx+1} Running Loss: {loss.item()}')

        # Adjust learning weights
        optimizer.step()

    tot_epoch_loss.append(sum(batch_loss)/len(batch_loss))
    print(f'Epoch Loss: {sum(batch_loss)/len(batch_loss)}')

  0%|          | 0/100 [00:00<?, ?it/s]

EPOCH: 0/100
BATCH 1 Running Loss: 6.573877334594727
BATCH 101 Running Loss: 5.596915245056152
BATCH 201 Running Loss: 4.44131326675415


  1%|          | 1/100 [00:18<30:10, 18.29s/it]

Epoch Loss: 5.3855435533343625
EPOCH: 1/100
BATCH 1 Running Loss: 3.694993019104004
BATCH 101 Running Loss: 3.295201063156128
BATCH 201 Running Loss: 2.768019676208496


  2%|▏         | 2/100 [00:36<29:55, 18.33s/it]

...

Epoch Loss: 0.12059677110387469
EPOCH: 98/100
BATCH 1 Running Loss: 0.0499076209962368
BATCH 101 Running Loss: 0.036566611379384995
BATCH 201 Running Loss: 0.10609625279903412


 99%|█████████▉| 99/100 [29:48<00:17, 17.99s/it]

Epoch Loss: 0.12550743781733062
EPOCH: 99/100
BATCH 1 Running Loss: 0.08940045535564423
BATCH 101 Running Loss: 0.07910189777612686
BATCH 201 Running Loss: 0.12410998344421387


100%|██████████| 100/100 [30:07<00:00, 18.07s/it]

Epoch Loss: 0.12065001809090939


In [ ]:
torch.cuda.is_available()

True

In [ ]:
device

'cuda'